In [ ]:
from modules.data_gen import DataGenerator, get_data, real_u1
from modules.pinn import PINNModel, model1, model2, optm1

from modules.accuracy import AccuracyCalc
from modules.plotting import NNPlots

import os

In [ ]:
session = 4
data = get_data(sess=session)
koefs_list = data['big_range']

model = PINNModel(model1(), optm=optm1())
dg = DataGenerator(data['x'], data['y'], model.predict, real_u1)

grid = (70, 70)

dataset = dg.inner_pairs(grid), dg.border_pairs(grid)

acc = AccuracyCalc(dg, grid)

In [3]:
# %%capture
# to supress output

existing_koefs = []
mse_loss_record = []
max_loss_record = []
train_loss_record = []

output = False

for koef in koefs_list:
    if os.path.exists(DataGenerator.name(koef, session)):
        model.load(dg.name(koef, session))
        acc.update_predicitons((70, 70))

        existing_koefs.append(koef)
        max_loss_record.append(acc.maxe())
        mse_loss_record.append(acc.mse())
        train_loss_record.append(model.loss(koef, *dataset))
        if output:
            print(f'mse_loss: {mse_loss_record[-1]}')
            print(f'max_loss: {max_loss_record[-1]}')
            print(f'model loss: {model.loss(koef, *dataset)}')
            print()

In [ ]:
plotter = NNPlots(dg, grid)

plotter.plot_error(existing_koefs, mse_loss_record, "MSE")
plotter.plot_error(existing_koefs, max_loss_record, "MAX")
plotter.plot_error(existing_koefs, train_loss_record, "Loss")

In [ ]:
# plotter.plot2d_contour(color_map="gnuplot2")
plotter.plot2d_contour()